In [2]:
import numpy as np
import torch

def random_vec_384(count):
    return torch.tensor(np.ones((count, 384)), dtype=torch.float32)

In [3]:
print(random_vec_384(3))

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.l1 = nn.Linear(384, 256)
        self.l2 = nn.Linear(256, 4)

    def forward(self, x):
        x = torch.flatten(x)
        x = F.relu(self.l1(x))
        z = self.l2(x)
        return z

In [5]:
class Decoder(torch.nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.l1 = nn.Linear(4, 256)
        self.l2 = nn.Linear(256, 384)
    def forward(self, z):
        z = F.relu(self.l1(z))
        x_hat = torch.sigmoid(self.l2(z))
        return x_hat

In [6]:
class Autoencoder(torch.nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
    def forward(self, x):
        z = self.enc(x)
        x_hat = self.dec(z)
        return x_hat

In [7]:
def loss(x, x_hat):
    # squared loss function
    return ((x - x_hat) ** 2).sum()

In [8]:
import wandb
import tqdm
import torch.optim

def train_vae(vae, data, epochs):
    # start a new wandb run to track this script
    wandb.init(
        # set the wandb project where this run will be logged
        project="dna2vec-vae",

        # track hyperparameters and run metadata
        config={
            "architecture": "VAE",
            "epochs": epochs
        }
    )
    optim = torch.optim.Adam(vae.parameters())
    wandb.watch(vae, loss, log="all", log_freq=1)
    for epoch in tqdm.tqdm(range(epochs)):
        for x in data:
            x = x.to('cuda')
            optim.zero_grad()
            x_hat = vae(x)
            L = loss(x, x_hat)
            wandb.log({"loss": L, "epoch": epoch})
            L.backward()
            optim.step()
    wandb.finish()
    return vae

In [10]:
vae = Autoencoder().to('cuda')
data = random_vec_384(100)
vae = train_vae(vae, data, 30)

100%|██████████| 30/30 [01:06<00:00,  2.23s/it]
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,29
loss,0.0


In [13]:
torch.save(vae.state_dict(), 'autoencoder.pth')

In [14]:
invec = torch.ones((1, 384)).to('cuda')
enc = vae.enc(invec)
print(enc)
print(vae.dec(enc))
print("Loss:", loss(invec, vae(invec)))

invec = torch.zeros((1, 384)).to('cuda')
enc = vae.enc(invec)
print(enc)
print(vae.dec(enc))
print("Loss:", loss(invec, vae(invec)))

tensor([  3.8305,  -7.3508, -17.2651, -23.0824], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000

AttributeError: 'NoneType' object has no attribute '_log'